### Tutorial of this notebook: https://towardsdatascience.com/data-science-skills-web-scraping-using-python-d1a85ef607ed

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import csv

In [2]:
# define the url that you are scraping
url = 'http://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'

Make the connection to the webpage and we can parse the html using BeautifulSoup

In [4]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(url)

# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
print(soup)

<!-- Template Name: League Table page
-->
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie ie6 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie ie7 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie ie8 no-js" lang="en-GB"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie ie9 no-js" lang="en-GB"> <![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en-GB">
<!--<![endif]-->
<!-- the "no-js" class is for Modernizr. -->
<head id="live2-fasttrack-com"><link data-minify="1" href="https://www.fasttrack.co.uk/wp-content/cache/min/1/30803e3d5151bb834fe97d4b7290956d.css" rel="stylesheet"/>
<meta charset="utf-8"/>
<!-- Always force latest IE rendering engine (even in intranet) & Chrome Frame -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>
        League table - Fast Track    </title>
<meta content="League table - Fast Track" name="title"/>
<meta content="" name="description"/>
<meta content="" name="keyword"/>
<meta con

As all of the results are contained within a table, we can search the soup object for the table using the find method. We can then find each row within the table using the find_all method.

In [10]:
# find results within table
table = soup.find('table', attrs={'class': 'tableSorter2'})
results = table.find_all('tr')
print('Number of results', len(results))

Number of results 101


If we print the number of rows we should get a result of 101, the 100 rows plus the header.

In [14]:
print(table)

<table class="tableSorter2">
<thead>
<tr>
<th>Rank</th>
<th>Company</th>
<th class="">Location</th>
<th class="no-word-wrap">Year end</th>
<th class="" style="text-align:right;">Annual sales rise over 3 years</th>
<th class="" style="text-align:right;">Latest sales £000s</th>
<th class="" style="text-align:right;">Staff</th>
<th class="">Comment</th>
<!--				<th>FYE</th>-->
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td><a href="https://www.fasttrack.co.uk/company_profile/revolut-2/"><span class="company-name">Revolut</span></a>Digital banking services provider</td>
<td>East London</td>
<td>Dec 18</td>
<td style="text-align:right;">507.56%</td>
<td style="text-align:right;">*58,300</td>
<td style="text-align:right;">700</td>
<td>Valued at $1.7bn in 2018 and reported to be raising an additional $500m this year that could value it at $5bn</td>
<!--						<td>Dec 18</td>-->
</tr>
<tr>
<td>2</td>
<td><a href="https://www.fasttrack.co.uk/company_profile/bizuma-3/"><span class="company-name">Bizu

There are 8 columns in the table containing: Rank, Company, Location, Year End, Annual Sales Rise, Latest Sales, Staff and Comments, all of which are interesting data that we can save.
This structure is consistent throughout all rows on the webpage (which may not always be the case for all websites!), and therefore we can again use the find_all method to assign each column to a variable.

In [15]:
# create and write headers to a list 
rows = []
rows.append(['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'])
print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments']]


In [27]:
result.find_all('td')

[<td>100</td>,
 <td><a href="https://www.fasttrack.co.uk/company_profile/dianomi-5/"><span class="company-name">Dianomi</span></a>Financial marketing platform</td>,
 <td>Central London</td>,
 <td>Dec 18</td>,
 <td style="text-align:right;">49.21%</td>,
 <td style="text-align:right;">*14,615</td>,
 <td style="text-align:right;">27</td>,
 <td>Works with eight of the top 10 global asset managers</td>]

In [24]:
# loop over results
for result in results:
    # find all columns per result
    data = result.find_all('td')
    # check that columns have data 
    if len(data) == 0: 
        continue
    
    # write columns to variables
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearend = data[3].getText()
    salesrise = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()
    
    # print('Company is', company)
    # Company is WonderblyPersonalised children's books
    # print('Sales', sales)
    # Sales *25,860

    # extract description from the name
    companyname = data[1].find('span', attrs={'class':'company-name'}).getText()    
    description = company.replace(companyname, '')
    
    # remove unwanted characters
    sales = sales.strip('*').strip('†').replace(',','')
    
    # go to link and extract company website
    url = data[1].find('a').get('href')
    page = urllib.request.urlopen(url)
    # parse the html using beautiful soup and store in variable 'soup'
    soup = BeautifulSoup(page, 'html.parser')
    # find the last result in the table and get the link
    try:
        tableRow = soup.find('table').find_all('tr')[-1]
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None
    
    # write each result to rows
    rows.append([rank, companyname, webpage, description, location, yearend, salesrise, sales, staff, comments])


print(rows)

[['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments'], ['1', 'Revolut', 'http://www.revolut.com', 'Digital banking services provider', 'East London', 'Dec 18', '507.56%', '58300', '700', 'Valued at $1.7bn in 2018 and reported to be raising an additional $500m this year that could value it at $5bn'], ['2', 'Bizuma', 'http://www.bizuma.com', 'B2B e-commerce platform', 'Central London', 'Mar 19', '315.18%', '26414', '114', 'Connects wholesale buyers and sellers from over 50 countries'], ['3', 'Global-e', 'http://www.global-e.com', 'Cross-border ecommerce solutions', 'Central London', 'Dec 18', '303.09%', '29297', '28', 'Its technology helps ecommerce retailers localise their websites in more than 200 markets'], ['4', 'Jungle Creations', 'http://www.junglecreations.com', 'Social media & ecommerce services', 'East London', 'Dec 18', '302.53%', '15972', '159', 'Launched the first-ever delivery-only re

In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame(rows)
df

,0,1,2,3,4,5,6,7,8,9
0,Rank,Company Name,Webpage,Description,Location,Year end,Annual sales rise over 3 years,Sales £000s,Staff,Comments
1,1,Revolut,http://www.revolut.com,Digital banking services provider,East London,Dec 18,507.56%,58300,700,Valued at $1.7bn in 2018 and reported to be ra...
2,2,Bizuma,http://www.bizuma.com,B2B e-commerce platform,Central London,Mar 19,315.18%,26414,114,Connects wholesale buyers and sellers from ove...
3,3,Global-e,http://www.global-e.com,Cross-border ecommerce solutions,Central London,Dec 18,303.09%,29297,28,Its technology helps ecommerce retailers local...
4,4,Jungle Creations,http://www.junglecreations.com,Social media & ecommerce services,East London,Dec 18,302.53%,15972,159,Launched the first-ever delivery-only restaura...
...,...,...,...,...,...,...,...,...,...,...
96,96,WorldRemit,http://www.worldremit.com,Currency transfer provider,Central London,Dec 18,50.26%,90965,615,"Is valued at more than $900m, having raised mo..."
97,97,E3D Online,http://www.e3d-online.com,3D printer developer,Oxfordshire,Apr 19,50.10%,6900,41,Has grown its exports to account for 75% of it...
98,98,Parentpay,http://www.parentpay.com,School payment services provider,Coventry,Nov 18,49.97%,30400,210,"Bought Dutch rival WIS in 2018, gaining access..."
99,99,Victor,http://www.flyvictor.com,Online private jet hire,Central London,Dec 18,49.56%,55100,100,Counts Sir Elton John among its investors


In [35]:
df.columns = ['Rank', 'Company Name', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Sales £000s', 'Staff', 'Comments']

In [47]:
df.drop(df.index[0], inplace=True)

In [51]:
df.set_index('Rank', inplace=True)

In [52]:
df

,Company Name,Webpage,Description,Location,Year end,Annual sales rise over 3 years,Sales £000s,Staff,Comments
Rank,,,,,,,,,
1,Revolut,http://www.revolut.com,Digital banking services provider,East London,Dec 18,507.56%,58300,700,Valued at $1.7bn in 2018 and reported to be ra...
2,Bizuma,http://www.bizuma.com,B2B e-commerce platform,Central London,Mar 19,315.18%,26414,114,Connects wholesale buyers and sellers from ove...
3,Global-e,http://www.global-e.com,Cross-border ecommerce solutions,Central London,Dec 18,303.09%,29297,28,Its technology helps ecommerce retailers local...
4,Jungle Creations,http://www.junglecreations.com,Social media & ecommerce services,East London,Dec 18,302.53%,15972,159,Launched the first-ever delivery-only restaura...
5,Oxford Nanopore Technologies,http://www.nanoporetech.com,DNA analysis technology,Oxford,Dec 18,251.87%,32500,439,Has raised £451m in funding and was valued at ...
...,...,...,...,...,...,...,...,...,...
96,WorldRemit,http://www.worldremit.com,Currency transfer provider,Central London,Dec 18,50.26%,90965,615,"Is valued at more than $900m, having raised mo..."
97,E3D Online,http://www.e3d-online.com,3D printer developer,Oxfordshire,Apr 19,50.10%,6900,41,Has grown its exports to account for 75% of it...
98,Parentpay,http://www.parentpay.com,School payment services provider,Coventry,Nov 18,49.97%,30400,210,"Bought Dutch rival WIS in 2018, gaining access..."
